## Numerics imports

In [ ]:
from functools import partial
import numpy as np
import adaptive
adaptive.notebook_extension()

## Plotting imports

In [ ]:
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {
          'backend': 'ps',
          'axes.labelsize': 13,
          'font.size': 13,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300,
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}', r'\usepackage{xfrac}'])

## Parameter and system definitions

In [ ]:
import scipy.constants
import cmath

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle=26, g_factor_left=0, g_factor_right=0,
                 mu=10,
                 alpha_middle=20, alpha_left=0, alpha_right=0,
                 Delta_left=1, Delta_right=1,
                 B_x=0,
                 B_y=0,
                 B_z=0,
                 phase=0,
                 T=0.0,
                 V=0)

params = dict(**constants,
              **params_raw)

syst_pars = dict(
    L_m=200,
    L_x=10,
    L_sc_up=200,
    L_sc_down=200,
    z_x=1300,
    z_y=0,
    a=10,
    parallel_curve=True,
    sawtooth=False,
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=True,
    k_x_in_sc=True,
    wraparound=True,
    current=False,
    ns_junction=False)

# Figure 2. Bandstructures

In [ ]:
def get_spectrum(momentum, syst_pars, params, nbands=40):
    import sns_system, spectrum
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    energies = spectrum.calc_spectrum(syst_at_k, dict(**params, k_x=momentum), k=nbands)[0]
    return np.sort(energies)

def spectrum_wrapper(momentum, z_y, B_x, phase):
    syst_pars['z_y'] = z_y
    params['B_x'] = B_x
    params['phase'] = phase
    return get_spectrum(momentum, syst_pars=syst_pars, params=params)

def abs_min_log_loss(xs, xy_scale, data):
    from adaptive.learner.learner1D import default_loss
    data = {k: np.log(np.abs(v).min()) for k, v in data.items()}
    return default_loss(xs, xy_scale, data)

B = 1
W = syst_pars['L_m']
combos = [(0, 0, 0), (W/2, 0, 0), (W, 0, 0), (0, B, np.pi), (W/2, B, np.pi), (W, B, np.pi)]

learners = [adaptive.Learner1D(partial(spectrum_wrapper, z_y=z_y, B_x=B_x, phase=phase),
                               bounds=[0, 1], loss_per_interval=abs_min_log_loss)
            for z_y, B_x, phase in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y', 'B_x', 'phi'], combos), strategy='npoints')

runner = adaptive.BlockingRunner(learner, lambda l: l.learners[0].npoints > 200)

In [ ]:
mapping = {(z_y, B_x, phase): learner for (z_y, B_x, phase), learner in zip(combos, learners)}

def plot(ax, key, color):
    data = mapping[key].data
    xs, ys = map(np.array, zip(*sorted(data.items())))
    return ax.plot(xs, ys, c=color)

fig, axs = plt.subplots(3, sharex=True, sharey=True, figsize=(fig_width, 2*fig_height))
(ax1, ax2, ax3) = axs

line2 = plot(ax1, (0, B, np.pi), 'red')[0]
line1 = plot(ax1, (0, 0, 0), 'blue')[0]

plot(ax2, (W/2, B, np.pi), 'red')
plot(ax2, (W/2, 0, 0), 'blue')

plot(ax3, (W, B, np.pi), 'red')
plot(ax3, (W, 0, 0), 'blue')

for i, ax in enumerate(axs):
    ax.set_ylabel(r'$E/\Delta$')
    
    # ylims and yticks
    ax.set_ylim(-0.5, 0.5)
    yvals = [-0.4, 0, 0.4]
    ylabels = [f'${x}$' for x in yvals]
    ax.set_yticks(yvals)
    ax.set_yticklabels(ylabels)
    
    # xlims and xticks
    xvals = [-0.1, 0.5, 1]
    xlabels = ['$0$', r'$\sfrac{1}{2a}$', r'$\sfrac{1}{a}$']
    ax.set_xticks(xvals)
    ax.set_xticklabels(xlabels)
    ax.set_xlim(0.3, 1)

    # text inside image
    label = 'abc'[i]
    ax.text(0.89, 0.8, f'$\mathrm{{({label})}}$', transform=ax.transAxes)
    
    z_ys = ['0', 'W/2', 'W']
    ax.text(0.05, 0.45, f'$z_y={z_ys[i]}$', transform=ax.transAxes)

ax1.legend((line1, line2), (r'$\phi=0$, $B=0$', r'$\phi=\pi$, $B \ne 0$'),
           loc='upper center', bbox_to_anchor=(0.5, 1.7),
           fancybox=True, shadow=True, ncol=1)

ax3.set_xlabel('$k_x$')

plt.savefig("paper/figures/bandstructures.pdf", bbox_inches="tight")
plt.show()

# Figure 3. Phase diagrams